In [24]:
from transformers import AutoTokenizer, AutoModelForPreTraining
import transformers
from transformers import Trainer
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader
from datasets import load_dataset
import numpy as np
import sklearn
import evaluate
import json

In [25]:
jnlpba = load_dataset('jnlpba')


print(jnlpba)
print(jnlpba['train'][0])
print(jnlpba['validation'][0])


Found cached dataset jnlpba (/Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 37094
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7714
    })
})
{'id': '1', 'tokens': ['IL-2', 'gene', 'expression', 'and', 'NF-kappa', 'B', 'activation', 'through', 'CD28', 'requires', 'reactive', 'oxygen', 'production', 'by', '5-lipoxygenase', '.'], 'ner_tags': [1, 2, 0, 0, 9, 10, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0]}
{'id': '1', 'tokens': ['Number', 'of', 'glucocorticoid', 'receptors', 'in', 'lymphocytes', 'and', 'their', 'sensitivity', 'to', 'hormone', 'action', '.'], 'ner_tags': [0, 0, 9, 10, 0, 7, 0, 0, 0, 0, 0, 0, 0]}


In [26]:
full_names_tags = {"O": "Outside", "B-DNA": "Gene Begin", "I-DNA": "Gene Inside", "B-PROTEIN": "Protein Begin", "I-PROTEIN": "Protein Inside", "B-CELL-LINE": "Cell Line Begin", "I-CELL-LINE": "Cell Line Inside", "B-CELL-TYPE": "Cell Type Begin", "I-CELL-TYPE": "Cell Type Inside", "B-DISEASE": "Disease Begin", "I-DISEASE": "Disease Inside"}

In [27]:
# model on which I want to fine tune on
# i go the dataset page and than I go to the leaderbord and see a model on the first place 
# i take this model because the leaderboard shows which model has the best scores on the dataset and i want to be state of the art
#SOTA model for NER on jnlpba is KeBioLM and achieves F1 82

#I just realize that the model is possible to evaluate like its written here https://huggingface.co/sciarrilli/biobert-base-cased-v1.2-finetuned-ner#:~:text=it%20achieves%20the%20following%20results%20on%20the%20evaluation%20set%3A
#there is now absolute no need to fine tune the model. but it could be a good introduction into doing finetuning. I basically can just copy the variables which the guy displays
#can do the fine tuning with this variables - yes lets do it!

#--> found new sota for token classification https://paperswithcode.com/sota/token-classification-on-jnlpba
#here https://huggingface.co/chintagunta85/electramed-small-JNLPBA-ner?text=My+name+is+Wolfgang+and+I+live+in+Berlin on huggingface
#was fine tuned on https://huggingface.co/giacomomiolo/electramed_small_scivocab
#the reason why i wanna fine tune is because i want to figure out how this works for my seq2seq later
tokenizer = AutoTokenizer.from_pretrained("giacomomiolo/electramed_small_scivocab")

model = AutoModelForPreTraining.from_pretrained("giacomomiolo/electramed_small_scivocab")

# Set the training arguments
training_args = TrainingArguments(
    output_dir="test/",
    learning_rate=2e-05,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    seed=42,
    #adam is not available in HF lib this is why I take HF optimizer
    optim="adamw_hf",
    adam_beta1= 0.9,
    adam_beta2=0.999,
    adam_epsilon=0.00000001,
    lr_scheduler_type="linear",
    num_train_epochs=10,
)

#i need to check how to use optim_args

metric = evaluate.load("accuracy")

# Initialize the trainer
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=jnlpba['train'][:30000],
    #eval_dataset=jnlpba["validation"],
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

loading configuration file config.json from cache at /Users/maxhager/.cache/huggingface/hub/models--giacomomiolo--electramed_small_scivocab/snapshots/d66f0eec09a066c59818c942ff2c2eecaf2718ca/config.json
Model config ElectraConfig {
  "_name_or_path": "giacomomiolo/electramed_small_scivocab",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522

  0%|          | 0/10 [00:00<?, ?it/s]

KeyError: 2

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Found cached dataset yelp_review_full (/Users/maxhager/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

loading configuration file config.json from cache at /Users/maxhager/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


  0%|          | 0/125 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 1.0569350719451904, 'eval_accuracy': 0.565, 'eval_runtime': 263.3537, 'eval_samples_per_second': 3.797, 'eval_steps_per_second': 0.475, 'epoch': 3.0}
{'train_runtime': 12011.2327, 'train_samples_per_second': 0.25, 'train_steps_per_second': 0.031, 'train_loss': 0.9693538411458333, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=0.9693538411458333, metrics={'train_runtime': 12011.2327, 'train_samples_per_second': 0.25, 'train_steps_per_second': 0.031, 'train_loss': 0.9693538411458333, 'epoch': 3.0})

#alternatively if this is not working now than I can just take the already pretrained model and make predictions with it and store my results somewhere. 

#would be cool to have some kind of dashboard where I can track all the things i still need to do and where I am standing. i.e. a doc/board where i list all the models i wanna train and how 

general spoken i have 